<a href="https://colab.research.google.com/github/LeoFernanndes/notebooks/blob/master/stacking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd


url = 'https://raw.githubusercontent.com/LeoFernanndes/datasets/master/titanic_%20train.csv' 
titanic_df = pd.read_csv(url)

columns_order = ['PassengerId', 'Name', 'Fare', 'Sex', 'Age', 'Pclass', 'SibSp', 'Parch', 'Embarked', 'Cabin', 'Ticket', 'Survived']
titanic = titanic_df.reindex(columns_order, axis= 1)

# training columns

colunas = ['Fare', 'Sex', 'Age', 'Pclass', 'SibSp', 'Parch', 'Embarked', 'Cabin', 'Ticket']


In [0]:
# filling in the gaps

for column in titanic.columns:
  if titanic[column].dtype != 'object':
      titanic[column].fillna(titanic[column].mean(), inplace= True)
  else:
      titanic[column] = titanic[column].astype('str')
      titanic[column].fillna(titanic[column].mode(), inplace= True)


from sklearn.preprocessing import LabelEncoder


le = LabelEncoder()
for column in titanic.columns:
  if titanic[column].dtype == 'object':
    titanic[column] = le.fit_transform(titanic[column])


titanic_train = titanic.iloc[0: 500]
titanic_test = titanic.iloc[500: ]


x_train = titanic_train[colunas]
y_train = titanic_train['Survived']


x_test = titanic_test[colunas]
y_test = titanic_test['Survived']

In [0]:
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.metrics import f1_score, accuracy_score

df_resultados = pd.DataFrame({'PassengerId': titanic_test['PassengerId']})
accuracy_list = [] 
f1_list = []

n = 100
for iter in range(n):
  
  RF_clf = RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                        max_depth=None, max_features='auto', max_leaf_nodes=None,
                        min_impurity_decrease=0.0, min_impurity_split=None,
                        min_samples_leaf=1, min_samples_split=2,
                        min_weight_fraction_leaf=0.0, n_estimators=100,
                        n_jobs=None, oob_score=False, random_state=None,
                        verbose=0, warm_start=False)

  RF_clf.fit(x_train, y_train)
  y_pred = RF_clf.predict(x_test)
  
  accuracy = accuracy_score(y_pred, y_test)
  accuracy_list.append(accuracy)

  f1 = f1_score(y_pred, y_test)
  f1_list.append(f1)

  dataframe = pd.DataFrame({iter: y_pred}, index= titanic_test.index)

  df_resultados = pd.concat([df_resultados, dataframe], axis= 1)

df_resultados['Final'] = df_resultados.drop(['PassengerId'], axis= 1).mode(axis= 1)[0]
df_resultados['Porcentagem'] = df_resultados.drop(['PassengerId'], axis= 1).mean(axis= 1)
accuracy_serie = pd.Series(accuracy_list).sort_values(ascending= True)
f1_serie = pd.Series(f1_list).sort_values(ascending= True)

In [8]:
final_accuracy = accuracy_score(df_resultados['Final'], y_test)
final_f1 = f1_score(df_resultados['Final'], y_test)

final_accuracy, accuracy_serie.mean(), final_f1, f1_serie.mean()

(0.8260869565217391,
 0.8230179028132991,
 0.7571428571428571,
 0.7547716958533652)

In [11]:
df_resultados.loc[(df_resultados['Porcentagem'] < 0.75) & (df_resultados['Porcentagem'] > 0.25)]

,PassengerId,0,1,2,3,4,5,6,7,8,9,Final,Porcentagem
503,504,0,1,0,1,0,0,0,0,0,1,0.0,0.272727
567,568,0,1,0,0,1,0,1,1,0,0,0.0,0.363636
621,622,0,1,1,1,0,0,1,0,0,0,0.0,0.363636
667,668,1,1,0,0,1,0,0,1,0,0,0.0,0.363636
741,742,0,1,0,1,1,0,1,0,0,0,0.0,0.363636
752,753,0,0,1,1,1,1,1,0,1,1,1.0,0.727273
756,757,0,1,1,0,1,1,1,1,1,0,1.0,0.727273
832,833,0,1,0,0,0,0,1,0,1,0,0.0,0.272727
885,886,1,0,0,0,1,0,1,1,0,1,0.0,0.454545
888,889,0,1,1,1,0,0,0,1,0,0,0.0,0.363636
